In [ ]:
!pip install transformers datasets evaluate sentencepiece sacremoses --quiet --upgrade

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 45.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 19.7 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cuda-cupti-cu12 12.5.82 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-nvrtc-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you hav

In [ ]:
import re
import gc
from datasets import load_dataset, Dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer


KeyboardInterrupt: 

In [ ]:

textbook_url = "https://huggingface.co/datasets/zxvix/MedicalTextbook/resolve/main/Anatomy_Gray/train-00000-of-00001.parquet"
df_textbook = pd.read_parquet(textbook_url)

disease_url = "https://huggingface.co/datasets/ares1123/disease_symtoms/resolve/main/disease_sympts_prec_full.csv"
df_disease = pd.read_csv(disease_url)

dataset_textbook = Dataset.from_pandas(df_textbook)
dataset_disease = Dataset.from_pandas(df_disease)


In [ ]:
# 🔹 Clean textbook entries
def clean_textbook_entry(example):
    text = example.get("text", "")
    text = re.sub(r"\[.*?\]", "", text)
    text = re.sub(r"\(.*?\)", "", text)
    text = re.sub(r"<.*?>", "", text)
    text = re.sub(r"\s+", " ", text).strip()
    return {"text": text}

textbook_cleaned = dataset_textbook.map(clean_textbook_entry, remove_columns=dataset_textbook.column_names)



In [ ]:
# 🔹 Format disease entries
def format_disease_entry(example):
    disease = example.get("Disease", "").strip()
    symptoms = example.get("Symptoms", "")
    precautions = example.get("Precautions", "")
    symptoms = ", ".join(symptoms) if isinstance(symptoms, list) else str(symptoms)
    precautions = ", ".join(precautions) if isinstance(precautions, list) else str(precautions)
    return {
        "text": f"Disease: {disease}. Symptoms include: {symptoms}. Precautions include: {precautions}."
    }

disease_formatted = dataset_disease.map(format_disease_entry, remove_columns=dataset_disease.column_names)



In [ ]:
#  Combine datasets
combined_texts = textbook_cleaned["text"] + disease_formatted["text"]
combined_dataset = Dataset.from_list([{"text": t} for t in combined_texts])


In [ ]:
#  Tokenization
model_name = "microsoft/BioGPT"
tokenizer = AutoTokenizer.from_pretrained(model_name)

def tokenize(example):
    tokens = tokenizer(
        example["text"],
        truncation=True,
        padding="max_length",
        max_length=256
    )
    tokens["labels"] = tokens["input_ids"][:]
    return tokens

tokenized_dataset = combined_dataset.map(tokenize, batched=True, remove_columns=["text"])


# 🔹 Load model
model = AutoModelForCausalLM.from_pretrained(model_name)

In [ ]:
# 🔹 Training Arguments
training_args = TrainingArguments(
    output_dir="./biogpt-medical-autocomplete",
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    num_train_epochs=3,
    logging_dir="./logs",
    save_strategy="epoch",
    fp16=True,
    report_to="none"
)


In [ ]:
# 🔹 Trainer Setup
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    tokenizer=tokenizer,
)


trainer.train()


In [ ]:
# 🔹 Save
trainer.save_model("./biogpt-medical-autocomplete")
tokenizer.save_pretrained("./biogpt-medical-autocomplete")

# 🔹 Cleanup
del model, tokenized_dataset, combined_dataset
gc.collect()


In [ ]:
!pip install huggingface_hub
from huggingface_hub import login
login('')  # paste your HF token

from transformers import AutoModelForCausalLM

AutoModelForCausalLM.from_pretrained("biogpt-medical-autocomplete").push_to_hub("ezhdeha/biogpt-medical-autocomplete")


In [ ]:
from transformers import AutoTokenizer

model_dir = "./biogpt-medical-autocomplete"
AutoTokenizer.from_pretrained(model_dir).push_to_hub("ezhdeha/biogpt-medical-autocomplete")
